In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
image, label = train_data[0]

In [9]:
image.size()

torch.Size([3, 32, 32])

In [12]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [13]:
class NeuralNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x): # neural network logic
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')
    
    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Loss:  {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss:  2.1410
Training epoch 1...
Loss:  1.6763
Training epoch 2...
Loss:  1.4924
Training epoch 3...
Loss:  1.3715
Training epoch 4...
Loss:  1.2742
Training epoch 5...
Loss:  1.1878
Training epoch 6...
Loss:  1.1145
Training epoch 7...
Loss:  1.0555
Training epoch 8...
Loss:  1.0063
Training epoch 9...
Loss:  0.9602
Training epoch 10...
Loss:  0.9190
Training epoch 11...
Loss:  0.8791
Training epoch 12...
Loss:  0.8458
Training epoch 13...
Loss:  0.8152
Training epoch 14...
Loss:  0.7825
Training epoch 15...
Loss:  0.7561
Training epoch 16...
Loss:  0.7305
Training epoch 17...
Loss:  0.7029
Training epoch 18...
Loss:  0.6757
Training epoch 19...
Loss:  0.6505
Training epoch 20...
Loss:  0.6287
Training epoch 21...
Loss:  0.6075
Training epoch 22...
Loss:  0.5836
Training epoch 23...
Loss:  0.5604
Training epoch 24...
Loss:  0.5415
Training epoch 25...
Loss:  0.5170
Training epoch 26...
Loss:  0.4981
Training epoch 27...
Loss:  0.4777
Training epoch 28...
Loss:  0.

In [16]:
torch.save(net.state_dict(), 'trained_net.pth')

In [17]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

/var/folders/hp/ybg635_s0pb1c3jrbykg25vm0000gn/T/ipykernel_29740/1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.p

<All keys matched successfully>

In [20]:
correct = 0
total = 0
net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 69.54%


In [25]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['./images/cats/1.png', './images/horse/1.png']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: cat
Prediction: horse
